In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

importing libraries

In [ ]:
#import modules
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data=pd.read_csv("../input/logistic-regression-heart-disease-prediction/framingham_heart_disease.csv")
data.head()




**Data Exploration**

In [ ]:
data.shape

In [ ]:
data.describe().T

*Checking NULL values in the dataset*

In [ ]:
data.isnull().sum()

We will eliminate the Null Values by droping the rows having the NULL values

In [ ]:
data.dropna(axis=0,inplace=True)
data.isnull().sum()

*Plotting Data w.r.t. the target class*

In [ ]:
print(data.TenYearCHD.value_counts())
sns.countplot(x='TenYearCHD',data=data,palette='hls')
plt.show()

In [ ]:
sns.scatterplot(data=data,x="diaBP",y="heartRate",hue="TenYearCHD",size=50,legend=False)

In [ ]:
healthy=len(data[data['TenYearCHD']==0])
diseased=len(data[data['TenYearCHD']==1])
Percent_of_healthy = healthy/(healthy+diseased)
Percent_of_diseased= diseased/(healthy+diseased)
print("Percentage of healthy people is : ",Percent_of_healthy*100)
print("Percentage of diseased people is : ",Percent_of_diseased*100)

 From the above plots , we observe that our dataset is unbalanced.

We will balance it using various methods during fitting the model.Balancing is important as logistic regression on imbalanced dataset will give biased results.

**Feature Selection**

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn')

From the above correlation matrix we can find a very high correlation between currentSmoker and cigsPerDay,hence we can drop any one of them.

Also,education does not play any important role in heart attack hence we can also drop that column.

In [ ]:
data=data.drop('education',axis=1)

data=data.drop('cigsPerDay',axis=1)

In [ ]:
data.groupby('TenYearCHD').mean()

As diaBP,BMI,heartRate have values in similar zones and almost equal to one another for 1 or 0 values for TenYearCHD hence they can be dropped.

In other words,these 3 have low variance for categoreies of TenYearCHD

In [ ]:
data= data.drop(['diaBP','BMI','heartRate'],axis=1)

In [ ]:
data.head()

In [ ]:
x = data.loc[:,data.columns != 'TenYearCHD']
y = data.loc[:,data.columns == 'TenYearCHD']

**Dataset Balancing**

> Random UnderSampling

This method seeks to randomly select and remove samples from the majority class, consequently reducing the number of examples in the majority class in the transformed data.

“In random under-sampling (potentially), vast quantities of data are discarded. This can be highly problematic, as the loss of such data can make the decision boundary between the minority and majority instances harder to learn, resulting in a loss in classification performance.”

Using this approach is effective in situations where the minority class has a sufficient amount of examples despite the severe imbalance. On the other hand, it is always important to consider the prospects of valuable information being deleted as we randomly remove them from our data set since we have no way to detect or preserve the examples that are information rich in the majority class.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [ ]:
# instantiating the random undersampler
rus = RandomUnderSampler() 
# resampling X, y
X_rus, y_rus = rus.fit_resample(x, y)
# new class distribution
print(Counter(y_rus))

**Implementing the model**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.3, random_state=0)
logreg = LogisticRegression(class_weight="balanced",max_iter=1000)
logreg.fit(X_train, y_train)

**Model Evaluation**

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cnf = confusion_matrix(y_test, y_pred)


fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(cnf)
ax.grid(False)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
ax.set_ylim(1.5, -0.5)
for i in range(2):
    for j in range(2):
        ax.text(j, i, cnf[i, j], ha='center', va='center', color='red')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
mae = mean_absolute_error(y_test,y_pred)
mse= mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

Mean Absolute Error

In [ ]:
print(mae)


Mean Squared error

In [ ]:
print(mse)


R2 score

In [ ]:
print(r2)